Import des packages

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

#### Import de la base de données sur les incidents par arme à feu aux Etats-Unis entre 2013 et 2018.

In [2]:
#base = pd.read_csv("stage3.csv") ; import depuis google drive à la place

In [3]:
url="https://drive.google.com/file/d/1GGOLMc_Ow9yZC9sICegPegDggQuHOD3t/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
base = pd.read_csv(url)

In [4]:
pd.set_option('display.max_columns', None)
base.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


#### Création de fonctions permettant de transformer les colonnes codées pour comporter des dictionnaires et des listes.

In [5]:
import re

In [6]:
def convert_to_dict(value):
    if pd.isna(value):
        return value

    if '||' not in str(value):

        #some are corrupted: '|' instead of '||'
        if '|' not in str(value):

            result_dict = {}
            #Some are corrupted : 1: instead of ::
            if '::' in str(value):
                key, val = re.split(r'::?|:', str(value))
                result_dict[int(key)] = val
            elif ':' in str(value):
                key, val = value.split(':', 1)
                result_dict[int(key)] = val
            else:
                result_dict={0:value}
        
        else: 
            pairs = str(value).split('|')
            pairs[1].strip("'])")

            result_dict = {}
            for pair in pairs:
                #Some are corrupted : 1: instead of ::
                if '::' in pair:
                    key, val = re.split(r'::?|:', str(pair))
                    result_dict[int(key)] = val
                elif ':' in pair:
                    key, val = pair.split(':', 1)
                    result_dict[int(key)] = val
                else:
                    key, val=0,pair
                    result_dict[key]=val

        return result_dict

    if '||' in str(value):

        pairs = re.split(r'\|\|?\|', str(value))

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = re.split(r'::?|:', str(pair))
                result_dict[int(key)] = val
            elif ':' in pair:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
            else:
                key, val=0,pair
                result_dict[key]=val
        return result_dict


updated_base = base.copy()
list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict)
updated_base.head()

/tmp/ipykernel_57894/3908154133.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict)


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: 'Male'},0::Bernard Gillis,NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...",0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}",0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}",0::Danielle Imani Jameison||1::Maurice Eugene ...,{3: 'Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [7]:
def convert_to_list(value):
    if pd.isna(value):
        return value

    if '||' in str(value):
        liste = value.split('||')
        return liste


list_of_list_columns = ['incident_characteristics', 'sources']
updated_base[list_of_list_columns] = updated_base[list_of_list_columns].applymap(convert_to_list)
updated_base.head()


/tmp/ipykernel_57894/3406849889.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  updated_base[list_of_list_columns] = updated_base[list_of_list_columns].applymap(convert_to_list)


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: 'Male'},0::Bernard Gillis,NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...",0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}",0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}",0::Danielle Imani Jameison||1::Maurice Eugene ...,{3: 'Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0


### Création d'indicatrices pour pouvoir accéder à l'information disponible dans des listes

In [8]:
def make_indic1(var,description,new_var):
    list_indic=[]

    for i in updated_base[var]:
        if i is not None:
            if description in str(i) :
                list_indic.append(1)
            else:
                list_indic.append(0)
        else:
            list_indic.append(0)
        

    updated_base[new_var]=list_indic

In [9]:
carac=[("incident_characteristics","uicide","indic_suicide"),
    ("incident_characteristics","efensive","indic_defens"),
    ("incident_characteristics", "chool", "indic_school"),
    ("incident_characteristics","Home Invasion", "indic_home_inv"),
    ("incident_characteristics","ccident","indic_accident")]
    #excluding the first letter of the word because of the case-sensitivity

for var,description,new_var in carac:
    make_indic1(var,description,new_var)

updated_base.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: 'Male'},0::Bernard Gillis,NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0,1,0,0,0,1
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...",0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0,1,0,0,0,1
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}",0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0,1,0,0,0,1
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}",0::Danielle Imani Jameison||1::Maurice Eugene ...,{3: 'Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0,1,0,0,0,1


Export de la base en csv pour pouvoir être utilisée dans les statistiques descriptives dans le fichier stat_desc_gun_violence.ipynb.

In [10]:
updated_base.to_csv('updated_base.csv')

### Création d'une base de données comprenant les informations relatives aux suspects

In [11]:
data_suspects=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Subject-Suspect':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_sus']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_sus']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_sus']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['gender_sus']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['gender_sus']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_sus']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_sus']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
                
                if type(updated_base['participant_relationship'][index])==dict:
                    if key in updated_base['participant_relationship'][index].keys():
                        result_dict['relation']=updated_base['participant_relationship'][index][key]
                else:
                    result_dict['relation']=np.nan


                data_suspects.append(result_dict)

    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Subject-Suspect':
            result_dict={'id':index}

            if type(updated_base['participant_age_group'][index])==dict:
                if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_sus']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_sus']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_sus']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['gender_sus']=updated_base['participant_gender'][index][key]
            else:
                result_dict['gender_sus']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_sus']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_sus']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan

            if type(updated_base['participant_relationship'][index])==dict:
                if key in updated_base['participant_relationship'][index].keys():
                    result_dict['relation']=updated_base['participant_relationship'][index][key]
            else:
                result_dict['relation']=np.nan
                
            data_suspects.append(result_dict)

    index+=1


df_suspects=pd.DataFrame(data_suspects)

In [12]:
df_suspects.head()

,id,age_group_sus,gender_sus,status_sus,gun_type,gun_stolen,relation,age_sus
0,0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Adult 18+,Male,"Injured, Unharmed, Arrested",Unknown,Unknown,NaN,25
3,2,Adult 18+,Male,"Unharmed, Arrested",Unknown,Unknown,NaN,31
4,3,Adult 18+,Male,Killed,NaN,NaN,NaN,33


On obtient ici un dataframe permettant d'obtenir l'âge, le genre, le status et l'arme de tous les participants aux incidents caractérisés comme 'Subject-Suspect' afin de pouvoir faire des statistiques descriptives. La colonne 'id' corresponds à l'index de l'incident concerné dans la base updated_base. La colonne 'relation' correspond à la relation du suspect à la victime, ou la circonstance de l'incident. 

In [13]:
df_suspects.to_csv('df_suspects.csv')

### Création d'une base de données similaire contenant toutes les informations sur les victimes

In [14]:
data_victimes=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Victim':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_vict']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_vict']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_vict']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['gender_vict']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['gender_vict']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_vict']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_vict']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
            
                
                data_victimes.append(result_dict)


    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Victim':
            result_dict={'id':index}
            if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_vict']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_vict']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_vict']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['gender_vict']=updated_base['participant_gender'][index][key]
            else:
                result_dict['gender_vict']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_vict']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_vict']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan


            data_victimes.append(result_dict)

    index+=1


df_victim=pd.DataFrame(data_victimes)

In [15]:
df_victim.head()

,id,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen
0,0,Adult 18+,20,Male,Arrested,NaN,NaN
1,0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,Adult 18+,20,Male,Killed,NaN,NaN


In [16]:
df_victim.to_csv('df_victim.csv')

### Création d'indicatrices sur les variables d'intérêt identifiées

Avoir des indicatrices sur ces sujets permettra d'obtenir plus facilement certaines informations clefs qui seront importantes lorsque l'on va croiser cette base de données avec des données extérieures

In [17]:
#création d'une base unique avec les informations sur les suspects et les victimes
updated_base=updated_base.reset_index()
merge_sus=pd.merge(updated_base, df_suspects, how='outer', left_on='index', right_on='id')
merge_tot=pd.merge(merge_sus, df_victim, how='outer', right_on='id', left_on='index')
merge_tot.head()

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,20,Male,Arrested,NaN,NaN
1,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: 'Male'},0::Bernard Gillis,NaN,"{0: 'Killed', 1:

In [18]:
def make_indic2(var, val1, new_name):
    list_indic=[]
    for i in data_indic[var]:
        
        if i!=None:
            if type(i)==str:
                if val1 in i :
                    list_indic.append(1)
                else:
                    list_indic.append(0)
            else:
                list_indic.append(np.nan)
        else:
            list_indic.append(np.nan)
    data_indic[new_name]=list_indic

In [19]:
data_indic=merge_tot.copy()

In [20]:
to_indic=[('age_group_sus', 'Adult 18+', 'sus_adulte'),
    ('age_group_vict', 'Adult 18+', 'vict_adulte'),
    ('gender_sus', 'Male', 'sus_male'),
    ('gender_vict', 'Male', 'vict_male'),
    ('status_sus', 'Injured', 'sus_injured'),
    ('status_sus', 'Arrested', 'sus_arrested'),
    ('status_sus', 'Unharmed', 'sus_unharmed'),
    ('status_sus', 'Killed', 'sus_killed'),
    ('status_vict', 'Injured', 'vict_injured'),
    ('status_vict', 'Arrested', 'vict_arrested'),
    ('status_vict', 'Unharmed', 'vict_unharmed'),
    ('status_vict', 'Killed', 'vict_killed'),
    ('gun_type', 'Handgun', 'handgun_1'),
    ('gun_type', 'Rifle', 'rifle_1'),
    ('gun_type', '9mm', '9mm_1'),
    ('gun_type', 'Shotgun', 'shotgun_1'),
    ('gun_type_y', 'Handgun', 'handgun_2'),
    ('gun_type_y', 'Rifle', 'rifle_2'),
    ('gun_type_y', '9mm', '9mm_2'),
    ('gun_type_y', 'Shotgun', 'shotgun_2')]

for col, val1, new_name in to_indic: 
    make_indic2(col, val1, new_name)

In [21]:
data_indic.head()

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun_1,rifle_1,9mm_1,shotgun_1,handgun_2,rifle_2,9mm_2,shotgun_2
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,20,Male,Arrested,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,

In [22]:
#taking care of the issue of the gun_type variable

def one_col(col1, col2, new_col):
    data_indic[new_col] = np.where(
        (data_indic[col1] == 1) | (data_indic[col2] == 1),  # If either column is 1
        1,
        np.where(
            (data_indic[col1] == 0) | (data_indic[col2] == 0),  # If either column is 0
            0,
            np.nan  # If both columns are NaN
        )
    )

In [23]:
gun_type_list=[('handgun_1', 'handgun_2', 'handgun'),
    ('rifle_1','rifle_2','rifle'),
    ('9mm_1', '9mm_2', '9mm'),
    ('shotgun_1','shotgun_2','shotgun')]

for col1, col2, new_col in gun_type_list:
    one_col(col1,col2, new_col)

In [24]:
del data_indic['9mm_1'], data_indic['9mm_2'], data_indic['rifle_1'], data_indic['rifle_2'], data_indic['handgun_1'], data_indic['handgun_2'], data_indic['shotgun_1'], data_indic['shotgun_2']

In [25]:
data_indic.head()

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,9mm,shotgun
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,20,Male,Arrested,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
1,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
4,1,460726

In [26]:
young=list(range(18,26))
young=[str(num) for num in young]

def indic_age(var, list1, new_name):
    list_indic=[]
    for i in data_indic[var]:
        
        if i!=None:
            if type(i)==str:
                if i in list1 :
                    list_indic.append(1)
                else:
                    list_indic.append(0)
            else:
                list_indic.append(np.nan)
        else:
            list_indic.append(np.nan)
    data_indic[new_name]=list_indic

indic_age('age_sus', young, 'young_sus')
indic_age('age_vict', young, 'young_vict')

In [27]:
col=list(data_indic.columns)
col

['index',
 'incident_id',
 'date',
 'state',
 'city_or_county',
 'address',
 'n_killed',
 'n_injured',
 'incident_url',
 'source_url',
 'incident_url_fields_missing',
 'congressional_district',
 'gun_stolen_x',
 'gun_type_x',
 'incident_characteristics',
 'latitude',
 'location_description',
 'longitude',
 'n_guns_involved',
 'notes',
 'participant_age',
 'participant_age_group',
 'participant_gender',
 'participant_name',
 'participant_relationship',
 'participant_status',
 'participant_type',
 'sources',
 'state_house_district',
 'state_senate_district',
 'indic_suicide',
 'indic_defens',
 'indic_school',
 'indic_home_inv',
 'indic_accident',
 'id_x',
 'age_group_sus',
 'gender_sus',
 'status_sus',
 'gun_type_y',
 'gun_stolen_y',
 'relation',
 'age_sus',
 'id_y',
 'age_group_vict',
 'age_vict',
 'gender_vict',
 'status_vict',
 'gun_type',
 'gun_stolen',
 'sus_adulte',
 'vict_adulte',
 'sus_male',
 'vict_male',
 'sus_injured',
 'sus_arrested',
 'sus_unharmed',
 'sus_killed',
 'vict_in

Maintenant que toutes les variables d'intérêt sont présentes dans le dataframe, on revient au format initial pour lequel il y avait une seule ligne par incident.

In [31]:
aggregated_data = data_indic.groupby('incident_id').agg(
    date=('date','first'), #using first to get the first non-null value
    state=('state','first'),
    city_or_county=('city_or_county','first'),
    address=('address','first'),
    n_killed=('n_killed', 'first'),
    n_injured=('n_injured','first'),
    incident_url=('incident_url','first'),
    source_url=('source_url','first'),
    incident_url_fields_missing=('incident_url_fields_missing','first'),
    congressional_district=('congressional_district','first'),
    gun_stolen=('gun_stolen_x','first'),
    gun_type_dict=('gun_type_x','first'),
    incident_characteristics=('incident_characteristics','first'),
    latitude=('latitude','first'),
    location_description=('location_description','first'),
    longitude=('longitude','first'),
    n_guns_involved=('n_guns_involved','first'),
    notes=('notes','first'),
    participant_age=('participant_age','first'),
    participant_age_group=('participant_age_group','first'),
    participant_gender=('participant_gender','first'),
    participant_name=('participant_name','first'),
    participant_relationship=('participant_relationship','first'),
    participant_status=('participant_status','first'),
    participant_type=('participant_type','first'),
    sources=('sources','first'),
    state_house_district=('state_house_district','first'),
    state_senate_district=('state_senate_district','first'),
    indic_suicide=('indic_suicide','first'),
    indic_defens=('indic_defens','first'),
    indic_school=('indic_school','first'),
    indic_home_inv=('indic_home_inv','first'),
    indic_accident=('indic_accident','first'),
    gun_type=('gun_type_y','first'), #even if multiple guns could be used in one incident, keeping one is already a good enough information
    relation=('relation', 'first'), #same reasoning
    sus_adulte=('sus_adulte', lambda x: 1 if x.sum() >= 1 else 0), #custom lambda function used to indicate the presence instead of counting the indicators (because it would give corrupted answers because of the merged data)
    vict_adulte=('vict_adulte', lambda x: 1 if x.sum() >= 1 else 0),
    sus_male=('sus_male', lambda x: 1 if x.sum() >= 1 else 0),
    vict_male=('vict_male', lambda x: 1 if x.sum() >= 1 else 0),
    sus_injured=('sus_injured', lambda x: 1 if x.sum() >= 1 else 0),
    sus_arrested=('sus_arrested', lambda x: 1 if x.sum() >= 1 else 0),
    sus_unharmed=('sus_unharmed', lambda x: 1 if x.sum() >= 1 else 0),
    sus_killed=('sus_killed', lambda x: 1 if x.sum() >= 1 else 0),
    vict_injured=('vict_injured', lambda x: 1 if x.sum() >= 1 else 0),
    vict_arrested=('vict_arrested', lambda x: 1 if x.sum() >= 1 else 0),
    vict_unharmed=('vict_unharmed', lambda x: 1 if x.sum() >= 1 else 0),
    vict_killed=('vict_killed', lambda x: 1 if x.sum() >= 1 else 0),
    handgun=('handgun', lambda x: 1 if x.sum() >= 1 else 0),
    rifle=('rifle', lambda x: 1 if x.sum() >= 1 else 0),
    Nine_mm=('9mm', lambda x: 1 if x.sum() >= 1 else 0),
    shotgun=('shotgun', lambda x: 1 if x.sum() >= 1 else 0),
    young_sus=('young_sus', lambda x: 1 if x.sum() >= 1 else 0),
    young_vict=('young_vict', lambda x: 1 if x.sum() >= 1 else 0)
)

aggregated_data.reset_index(inplace=True)

In [32]:
aggregated_data.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
0,92114,2014-01-01,Kentucky,Lexington,Sixth Street and Elm Tree Lane,0,1,http://www.gunviolencearchive.org/incident/92114,http://www.lex18.com/news/lexington-police-sta...,False,6.0,None,None,None,38.0507,None,-84.4829,NaN,None,None,{0: 'Adult 18+'},{0: 'Male'},None,None,{0: 'Injured'},{0: 'Victim'},None,NaN,NaN,0,0,0,0,0,None,None,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,92117,2014-01-01,Kentucky,Cynthiana,None,0,1,http://www.gunviolencearchive.org/incident/92117,https://www.wtvq.com/2014/01/01/harrison-sheri...,False,4.0,{0: 'Unknown'},{0: 'Unknown'},"[Shot - Wounded/Injured, Accidental Shooting, ...",38.4333,None,-84.3542,1.0,US-62 east of Cynthiana,{1: '22'},"{0: 'Adult 18+', 1: 'Adult 18+'}","{0: 'Male', 1: 'Male'}",0::Dustin Mullins||1::Timothy Turner,{1: 'Friends'},"{0: 'Injured', 1: 'Unharmed, Arrested'}","{0: 'Victim', 1: 'Subject-Suspect'}",[http://www.lex18.com/news/man-arrested-in-con...,78.0,27.0,0,0,0,0,1,None,Friends,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0
2,92119,2014-01-01,Kentucky,Louisville,38th and Broadway,0,1,http://www.gunviolencearchive.org/incident/92119,http://www.whas11.com/news/local/Person-injure...,False,3.0,None,None,None,38.2501,None,-85.8150,NaN,None,None,{0: 'Adult 18+'},{0: 'Male'},None,None,{0: 'Injured'},{0: 'Victim'},None,NaN,NaN,0,0,0,0,0,None,None,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,92122,2014-01-01,Virginia,Falls Church,North Washington Street,0,0,http://www.gunviolencearchive.org/incident/92122,http://www.nbcwashington.com/news/local/Two-So...,False,8.0,None,None,None,38.8847,None,-77.1672,NaN,None,None,None,None,None,None,{0: 'Unharmed'},"{0: 'Victim', 1: 'Subject-Suspect'}",None,53.0,35.0,0,0,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,92125,2014-01-01,Oklahoma,Lawton,Sheridan and Pollard,0,0,http://www.gunviolencearchive.org/incident/92125,http://www.kswo.com/story/24342150/lpd-investi...,False,4.0,None,None,None,34.6347,None,-98.4222,NaN,None,None,{0: 'Adult 18+'},"{0: 'Male', 1: 'Male'}",None,None,"{0: 'Unharmed', 1: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect'}",None,64.0,32.0,0,0,0,0,0,None,None,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,1083428,2018-03-30,Illinois,Centralia,E Noleman St and N Maple St,0,1,http://www.gunviolencearchive.org/incident/108...,https://mvnnews.wordpress.com/2018/04/02/centr...,False,15.0,{0: 'Unknown'},{0: 'Unknown'},None,38.5281,None,-89.1295,1.0,Known suspect sought;,None,{0: 'Adult 18+'},"{0: 'Male', 1: 'Male'}",None,None,"{0: 'Injured', 1: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect'}",None,107.0,54.0,0,0,0,0,0,None,None,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
239673,1083435,2018-03-20,West Virginia,Williamson,Vinson St,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.williamsondailynews.com/news/police...,False,3.0,{0: 'Unknown'},{0: 'Unknown'},"[Shots Fired - No Injuries, Brandishing/flouri...",37.6750,None,-82.2672,1.0,Multiple shots fired through door,{0: '18'},{0: 'Adult 18+'},{0: 'Male'},0::Armando Israel Elizondo,None,{0: 'Arrested'},{0: 'Subject-Suspect'},None,20.0,6.0,0,0,0,0,0,Unknown,None

In [33]:
#sorting the dataframe by date to get the same order as in the original dataframe
agg_sorted = aggregated_data.sort_values(by='date', ascending=True)
agg_sorted

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,None,None,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,None,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,None,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,0,0,0,0,0,None,None,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1
104318,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,None,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...",0::Damien Bell||1::Desmen Noble||2::Herman Sea...,None,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0,1,0,0,0,1,Unknown,None,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,0
100093,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,None,None,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,None,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: 'Male'},0::Bernard Gillis,None,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0,1,0,0,0,1,None,None,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1
104325,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,None,None,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,None,-104.8020,NaN,None,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}",0::Stacie Philbrook||1::Christopher Ratliffe||...,None,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0,1,0,0,0,1,None,None,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
104332,478948,2013-01-07,Oklahoma,Tulsa,6000 block of South Owasso,4,0,http://www.gunviolencearchive.org/incident/478948,http://usnews.nbcnews.com/_news/2013/01/07/163...,False,1.0,None,None,"[Shot - Dead (murder, accidental, suicide), Ho...",36.2405,Fairmont Terrace,-95.9768,NaN,None,"{0: '23', 1: '23', 2: '33', 3: '55'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Female', 2: 'Female', 3: 'Fe...",0::Rebeika Powell||1::Kayetie Melchor||2::Mist...,None,"{0

In [ ]:
agg_sorted.to_csv('df_gun_violence.csv')